# PAAI21

Image classification can be successfully solved by modern CNNs. However, there are still
a plethora of questions regarding how those models manage to extract and model general
features for large number of classes. A straightforward strategy is to focus on saliency i.e.,
the area of the image that has a maximal response w.r.t. the predicted class. The aim of
this project is to complement that idea by estimating the number of salient regions that
common image classifiers react to and measure the consistency of those regions
regarding the predicted class.

## Resnet18 - Training

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import skimage.transform
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision

from keras.optimizers import SGD
from matplotlib.pyplot import imshow
from PIL import Image
from torch.autograd import Variable
from torch import topk
from torchvision import models, datasets, transforms
from sklearn.model_selection import train_test_split

In [ ]:
# -------------------------------------
# Define paramters and other variables
# -------------------------------------

# Classes labels CIFAR-10
classes = ('plane',
           'auto',
           'bird',
           'cat',
           'deer',
           'dog',
           'frog',
           'horse',
           'ship',
           'truck')

# CIFAR1-10 parameters
input_size = 32
num_classes = 10

# Check for GPU/CPU to allocate tensor
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
 #   normalize,
])

# Download & transform CIFAR-10 datasets
train_full_dataset = datasets.CIFAR10("./data", train=True,
                                 transform=train_transform, download=True)

test_dataset = datasets.CIFAR10("./data", train=False,
                                transform=test_transform, download=True)

# Split datasets
train_num_samples = int(len(train_full_dataset) * 0.9) # 90%(Training set).
val_num_samples = int(len(train_full_dataset) * 0.1)   #10%(Validation set)

print("Division: ",train_num_samples, val_num_samples)
train_dataset, validation_dataset = random_split(train_full_dataset, [train_num_samples, val_num_samples])
print("Dataset lenght: ", len(train_dataset))

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=5000, shuffle=False)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.resnet = models.resnet18(pretrained=False, num_classes=10)

        self.resnet.conv1 = torch.nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.resnet.maxpool = torch.nn.Identity()

    def forward(self, x):
        x = self.resnet(x)
        x = F.log_softmax(x, dim=1)

        return x

In [ ]:
def train_model(model, train_loader, optimizer, epoch, verbose=False):
    model.train()
    total_loss = []
  
    for data, target in train_loader:
        data = data.cuda()
        target = target.cuda()
    
        optimizer.zero_grad()
        prediction = model(data)
        loss = F.nll_loss(prediction, target)
    
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())
    
        avg_loss = sum(total_loss) / len(total_loss)
        if (verbose):
            print("Training set: Epoch: {} Average Loss: {:.2f}".format(epoch, avg_loss))

    return avg_loss


def _get_classes_percentage(targets, predictions):
    num_classes = len(classes)
    num_samples = len(targets)

    # Init class lists
    correct_per_class = [0] * num_classes
    total_per_class = [0] * num_classes
    percentage_per_class = [0] * num_classes
    for i in range(num_samples):
        class_id = targets[i]
        if targets[i] == predictions[i]:
            correct_per_class[class_id] += 1
        total_per_class[class_id] += 1
    
    for i in range(num_classes):
        percentage_per_class[i] = 100 * (correct_per_class[i] / total_per_class[i])
    
    return percentage_per_class
            

def test_model(model, test_loader, verbose=False):
    model.eval()
    loss = 0
    correct = 0

    for data, target in test_loader:
        with torch.no_grad():
            data = data.cuda()
            target = target.cuda()

            prediction = model(data)
            loss += F.nll_loss(prediction, target, reduction="sum")

            prediction = prediction.max(1)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

            loss /= len(test_loader.dataset)
            percentage_correct = 100.0 * correct / len(test_loader.dataset)
            percentage_classes = _get_classes_percentage(target, prediction)

            if (verbose):
                print("Testing set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
                    loss, correct, len(test_loader.dataset), percentage_correct))
      
    return loss, percentage_correct, percentage_classes


def format_model_output(e, avg_loss, tloss, pct_correct, pct_classes):
    output  = "Epoch:{: <2} ".format(e)
    output += "TrainLoss:{: <4.2f} ".format(avg_loss)
    output += "TestLoss:{: <4.2f} ".format(tloss)
    output += "Accuracy:{: <5.2f}% ".format(pct_correct)

    num_classes = len(classes)
    for i in range(num_classes):
        output += "{}:{: <5.2f}% ".format(classes[i], pct_classes[i])
    
    return output


In [ ]:
def build_model(train_loader, test_loader):
    # Hyperparameters
    epochs=50
    lr=0.1
    
    model = Model()
    model = model.cuda()

    milestones = [25, 40]

    optimizer = torch.optim.SGD(
        model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=milestones, gamma=0.1)

    print("Start train/test resnet18!")
    for epoch in range(1, epochs + 1):
        avg_loss = train_model(model, train_loader, optimizer, epoch)
        loss, pct_correct, pct_classes = test_model(model, test_loader)
        output = format_model_output(epoch, avg_loss, loss, pct_correct, pct_classes)
        print(output)
 
        scheduler.step()

    torch.save(model.state_dict(), "PAAI21_CIFAR10_model.pt")
    
    return model, pct_correct, pct_classes

In [ ]:
model_base, pct_correct, pct_classes = build_model(train_loader, test_loader)

In [ ]:
def plot_results(pct_correct, pct_classes):
  ind = [x for x, _ in enumerate(classes)]

  fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
  fig.suptitle('Accuracy results')

  # Bar chart
  ax[0].bar(ind, pct_classes, width=0.8, color='#00cc00')
  ax[0].bar(ind, pct_classes, width=0.8, color='#00cc00')
  ax[0].set_ylabel("Percentage")
  ax[0].set_xlabel("Classes")
  ax[0].set_xticks(ind)
  ax[0].set_xticklabels(classes)
  ax[0].set_title("Accuracy % per class - resnet18 and CIFAR10")

  # Pie chart
  ax[1].pie([100 - pct_correct, pct_correct], labels = ['Error ' + "{:.2f}".format(100 - pct_correct) +
        '%', 'Accuracy ' + str(pct_correct) + '%'], colors=['red', '#00cc00'], startangle = 90)
  ax[1].set_title("Accuracy model - resnet18 and CIFAR10")
  plt.show()



In [ ]:
#{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
plot_results(pct_correct, pct_classes)

# Class activation maps

## Terminology
- **Receptive field:** The receptive field of a neuron is composed by all
  pixels in **X** input that influence it.

- **Convolutional units:** A convolutional layer contains units whose receptive
  fields cover a patch of the previous layer[1].

- **Softmax:** It is a function used to become an output score in a probablity
  distribution.

- **Global average pooling (GAP):** It is an operation that consists of take the
  averag of each feature map and the resulting vector is used to feed the
  softmax layer[2]. 

## Summary

1. The convolutional units of several CNNs layers behave as object detectors
  even object location is not given. So they have the ability to localize
  objects.

2. This ability is lost when fully-connected layers are used for classification.

3. we can replace fully-connected layers by GAP.

4. There is no parameter to optimize in the global average pooling,
  thus overfitting is avoided at this layer. So GAP acts as a regularizer.

5. We can modify GAP and use it in combination with a class called 
  **class activation mapping (CAM)** to retain this localization ability
  until the final layer.

6. Therefore a CNN trained on object categorization is successfully able to
  localize the discriminative regions for action classification.

7. A **class activation map (CAM)** for a particular category indicates the
  discriminative image regions used by the CNN to identify that category.

8. we can identify the importance of the image regions by projecting back
  the weights of the output layer on to the convolutional feature maps, a
  technique we call **class activation mapping**.

9. Normally, We perform GAP on the convulitional feature maps and this
  output feed a fully-connected layer that produces the final output, So the
  weighted sum of GAP output is used to generate the final output
  (e.g. category of something).
 
10. We can identify the importance of the image regions by projecting back
   the weights of the output layer on to the convolutional feature maps.
 
## CAM description

$M_c(x, y) = \sum_k w_k^c f_k(x,y)$
<br><br>
Where:<br>
$M_c$ is the class activation map for a class $c$<br>
$f_k(x,y)$ is the activation of unit $k$ in the last convolutional layer at spatial location $(x,y)$<br>
$F^k$ is the output of GAP on $f_k(x,y)$ , then $F^k=\sum_{x,y} f_k(x,y)$<br>
$w_k^c$ indicates the importance of $F^k$ for class $c$


## References:
[1] https://en.wikipedia.org/wiki/Convolutional_neural_network

[2] https://arxiv.org/pdf/1312.4400v3.pdf



In [ ]:
################################### WARNING!! LOAD EXTERNAL MODEL !!!! ###################################

# Load model
model_base = Model()
model_base.cuda()

#from google.colab import files
#uploaded = files.upload()

#### Kaggle ########
model_base.load_state_dict(torch.load("../input/paaimodel/PAAI21_CIFAR10_model.pt"))
####################

model_base.eval()
print("model loaded!")
################################################################################################

In [ ]:
class LayerFeatures():
    features=None
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)

    def hook_fn(self, module, input, output):
        self.features = ((output.cpu()).data).numpy()

    def remove(self): self.hook.remove()


def compute_CAM(feature_conv, class_weights):
    _, num_channels, h, w = feature_conv.shape
    CAM = np.zeros((h, w))
    i = 0
    for act_map in feature_conv[0]:
        CAM += act_map * class_weights[i]
        i+=1

    # Now we need to normalize our CAM in [0,1] range
    CAM = CAM - np.min(CAM)
    CAM = CAM / np.max(CAM)

    return CAM


def get_one_random_sample(test_dataset):
    num_total_imgs = len(test_dataset.data)
    random_index = random.randint(1, num_total_imgs)
    img = test_dataset.data[random_index]
    label = test_dataset.targets[random_index]

    return img, label

# Compute CAM

In [ ]:

def get_heatmaps(tensor, model):
    #tensor = test_transform(image)
    prediction_var = Variable((tensor.unsqueeze(0)).cuda(), requires_grad=True)

    model.cuda()
    model.eval()
    model._modules.keys()

    final_layer = model._modules.get("resnet").layer4[-1]
    activated_features = LayerFeatures(final_layer)

    prediction = model(prediction_var)
    pred_probabilities = F.softmax(prediction, dim=1).data.squeeze()
    activated_features.remove()

    # Indentify the predicted class
    value, index = topk(pred_probabilities, 1)

    # Get information from identified class 
    weight_softmax_params = list(model._modules.get('resnet').fc.parameters())
    weight_softmax = np.squeeze(weight_softmax_params[0].cpu().data.numpy())
    class_id = topk(pred_probabilities,1)[1].int()
    class_weights = weight_softmax[class_id]
    cam_img = compute_CAM(activated_features.features, class_weights)

    # As we can see, our CAM size does not match with the our
    # image. We need to resize our map and interpolate the values
    # according to our image
    heat_map = skimage.transform.resize(cam_img, tensor.shape[1:3])

    return cam_img, heat_map, index, value


def crop_preprocess(x, model):
    # The 5% of the highest values represent a one of the most
    # important values for classification. These values will be
    # for experiments modified.
    #print("myinfo:", type(x))
    cam_img, heat_map, index, value = get_heatmaps(x, model)
    percentile = 95
    h, w = heat_map.shape
    feature_thld = np.percentile(heat_map, percentile)
    heat_mask = heat_map.copy()
    bk_mask = heat_map.copy()

    heat_mask = np.where(heat_mask >= feature_thld, 0.0, 1.0)
    bk_mask = np.where(bk_mask >= feature_thld, 1.0, 0.0) # Replace value 
    x = np.multiply(x, heat_mask)
    x = x + bk_mask
    #x = x.cpu().numpy()
    x = x.to(device='cuda')
    #if x.is_cuda == True:
    #    print("I am in CUDA")
    #else:
    #    print("I am in CPU")
    #return x
    return x.float()

def show_sample_images(train_loader):
    num_imgs_toshow= 10
    data_iter = iter(train_loader)
    images, labels = data_iter.next()
    # convert images to numpy for display
    images = images.cpu().numpy()

    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(15, 5))
    for i in np.arange(num_imgs_toshow):
        ax = fig.add_subplot(2, num_imgs_toshow/2, i + 1, xticks=[], yticks=[])
        plt.imshow(np.transpose(images[i], (1, 2, 0)))

def vis_demo(original, cam, heat_map, crop):
    # Plot
    plt.figure()
    f, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 18)) 
    ax[0].imshow(original)
    ax[0].set_title("Original")
    ax[1].imshow(cam, alpha=0.5, cmap='jet')
    ax[1].set_title("Class activation map")
    ax[2].imshow(original)
    ax[2].imshow(heat_map, alpha=0.5, cmap='jet')
    ax[2].set_title("Heat map")
    #ax[3].imshow(crop.argmax())
    ax[3].imshow(np.transpose(crop.cpu(), (1, 2, 0)))
    ax[3].set_title("Cropped image")
    plt.show()
    

In [ ]:
# CAM
image, label =  get_one_random_sample(test_dataset)
image_tensor = test_transform(image)
cam_img, heat_map, index, value = get_heatmaps(image_tensor, model_base)

# CROP most relevant area(s)
crop_transformation = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    #transforms.Lambda(lambda x: crop_preprocess(x, model_base)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: crop_preprocess(x, model_base)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
])
cropped_image = crop_transformation(image)
print("IMAGE CLASS: ", classes[label])
print("PREDICTION: ", classes[index.tolist()[0]])
print("ACCURACY: {:.2f}%".format(value.tolist()[0] * 100))
vis_demo(image, cam_img, heat_map, cropped_image)

In [ ]:

crop_transformation_a1 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),#########################
    transforms.RandomHorizontalFlip(),##################################
    #transforms.Lambda(lambda x: crop_preprocess(x, model_base)),
    transforms.ToTensor(),##############################
    transforms.Lambda(lambda x: crop_preprocess(x, model_base)),####################
])

train_full_dataset_a1 = datasets.CIFAR10("./data", train=True,
                                 transform=crop_transformation_a1, download=True)

test_dataset_a1 = datasets.CIFAR10("./data", train=False,
                                transform=test_transform, download=True)

# Split datasets
train_num_samples_a1 = int(len(train_full_dataset_a1) * 0.9) # 90%(Training set).
val_num_samples_a1 = int(len(train_full_dataset_a1) * 0.1)   #10%(Validation set)
train_dataset_a1, validation_dataset_a1 = random_split(train_full_dataset_a1, [train_num_samples_a1, val_num_samples_a1])

# Loaders
train_loader_a1 = torch.utils.data.DataLoader(train_full_dataset_a1,
                                           batch_size=128, shuffle=True)
test_loader_a1 = torch.utils.data.DataLoader(test_dataset_a1,
                                          batch_size=5000, shuffle=False)

In [ ]:
show_sample_images(train_loader_a1)

In [ ]:
model_a1, pct_correct_a1, pct_classes_a1 = build_model(train_loader_a1, test_loader_a1)

In [ ]:
plot_results(pct_correct_a1, pct_classes_a1)